In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('ALS Book Recommendation System').getOrCreate()

In [0]:
ratingDF = spark.read.csv('/FileStore/tables/ratings.csv',header="true",inferSchema="true")

In [0]:
bookDF = spark.read.csv('/FileStore/tables/books.csv',header="true",inferSchema="true")

In [0]:
avgRatingDF = ratingDF.groupBy("book_id").avg("rating").orderBy("avg(rating)", ascending=False)

In [0]:
avgRatingDF.show()

+-------+------------------+
|book_id|       avg(rating)|
+-------+------------------+
|   7947| 4.820224719101123|
|   5207|              4.78|
|   6920|              4.78|
|   9566| 4.777777777777778|
|   8946| 4.774193548387097|
|   6361|              4.77|
|   3275|              4.77|
|   4483|              4.75|
|   5580|              4.75|
|   6590|              4.75|
|   1308|              4.74|
|   3628|              4.73|
|   8978|              4.73|
|   3753|              4.72|
|   8109|              4.71|
|   1788|               4.7|
|   9076| 4.695652173913044|
|   8663|4.6947368421052635|
|   9141|              4.69|
|    862|              4.68|
+-------+------------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import *

In [0]:
avgRatingDF.select("book_id", bround("avg(rating)", 2)).show()

+-------+----------------------+
|book_id|bround(avg(rating), 2)|
+-------+----------------------+
|   7947|                  4.82|
|   5207|                  4.78|
|   6920|                  4.78|
|   9566|                  4.78|
|   8946|                  4.77|
|   6361|                  4.77|
|   3275|                  4.77|
|   4483|                  4.75|
|   5580|                  4.75|
|   6590|                  4.75|
|   1308|                  4.74|
|   3628|                  4.73|
|   8978|                  4.73|
|   3753|                  4.72|
|   8109|                  4.71|
|   1788|                   4.7|
|   9076|                   4.7|
|   8663|                  4.69|
|   9141|                  4.69|
|    862|                  4.68|
+-------+----------------------+
only showing top 20 rows



In [0]:
ratDF = avgRatingDF.select("book_id",bround("avg(rating)",2)).withColumnRenamed("bround(avg(rating),2)","rating")

In [0]:
titleBookDF = bookDF.select("book_id","original_title")

In [0]:
avgBookTitleDF=titleBookDF.join(ratDF, on="book_id", how='left')

In [0]:
avgBookTitleDF.show()

+--------+--------------------+----------------------+
| book_id|      original_title|bround(avg(rating), 2)|
+--------+--------------------+----------------------+
|    5907|The Hobbit or The...|                  3.89|
|      34| The Fellowship o...|                  2.77|
|13335037|           Divergent|                  null|
|    7613|Animal Farm: A Fa...|                  4.15|
|       3|Harry Potter and ...|                  3.09|
|   77203|    The Kite Runner |                  null|
|       5|Harry Potter and ...|                  3.89|
|   48855|Het Achterhuis: D...|                  null|
| 2429135|Män som hatar kvi...|                  null|
|    2657|To Kill a Mocking...|                  3.65|
|    5470|Nineteen Eighty-Four|                  3.17|
|   41865|            Twilight|                  null|
|     960|    Angels & Demons |                  4.06|
| 2767052|    The Hunger Games|                  null|
|11870085|The Fault in Our ...|                  null|
| 6148028|

In [0]:
bookDF.orderBy("original_publication_year", ascending=True).select("original_title","original_publication_year").show(10)

+------------------------+-------------------------+
|          original_title|original_publication_year|
+------------------------+-------------------------+
|    The Lord of the R...|                     null|
|    Nancy Drew Comple...|                     null|
|                    null|                     null|
|                    null|                     null|
|                    null|                     null|
|                    null|                     null|
|                    null|                     null|
| 絶対彼氏 (Zettai Kar...|                     null|
|                    null|                     null|
|    Twilight: The Com...|                     null|
+------------------------+-------------------------+
only showing top 10 rows



In [0]:
bookDF.filter("original_publication_year is not null OR original_title is not null").show()

+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id| book_id|best_book_id| work_id|books_count|      isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+-----------

In [0]:
 filterBookDF=bookDF.filter("original_publication_year is not null and original_publication_year > 0")

In [0]:
filterBookDF.createOrReplaceTempView("books")

In [0]:
%sql
select original_publication_year, count(*) as count from books where original_publication_year > 1968 group by original_publication_year

original_publication_year,count
1988.0,89
1976.0,39
1979.0,48
1987.0,83
1978.0,48
2010.0,473
1993.0,115
2001.0,226
1984.0,67
1973.0,35


Databricks visualization. Run in Databricks to view.